![li_banner_1_blue.png](./OI_images/li_banner_1_blue.png)

# Proteox system integration with QCoDeS

The Proteox is controlled with the `oi.DECS` system control software.  This is designed to operate in an asynchronous fashion using WebSockets.

`QCoDeS` works well for `VISA` instruments that generally have synchronous behaviour.

It is possible to the `WAMP` messaging protocol directly from QCoDeS, but for simple (single threaded) instrument drivers that can result in time-out issues if the communication channel is unactive for some time.

An alternative approach is to make `oi.DECS` 'look like' a standard `VISA` instrument, and that approach is described below.

## DECS<->VISA

To accomplish this, a simple socket server (launched *via* `decs_visa.py`) is inserted between `QCoDeS` and `oi.DECS`.  A pair of queues objects are provided for inter-process communication between the socket server and the `WAMP` interface to the Proteox.

![DECS_VISA_img.PNG](./OI_images/DECS_VISA.jpg)

# Example of Proteox Control using QCoDeS driver

First, import the required python packages and Proteox oi.DECS QCoDeS driver, starting logging and create a Station.

In [1]:
import logging
import sys

import qcodes as qc
from qcodes.logger.logger import start_all_logging

sys.path.append("../../src/qcodes_contrib_drivers/drivers/OxfordInstruments/")
from Proteox import oiDECS

start_all_logging()
log = logging.getLogger()

station = qc.Station()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\abi.graham\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\abi.graham\.qcodes\logs\240226-13388-qcodes.log


**Initialise connection to oi.DECS software/Proteox.**

In [2]:
try:
    Proteox = oiDECS('Proteox')
    Proteox.timeout(15)
except Exception as err:
    log.critical(err)
    print(f'Connection failed: {err}')

Running on Windows-10-10.0.19045-SP0 - start subprocess without PIPEd output
Connected to: Oxford Instruments oi:DECS (serial:decs-506c7d, firmware:0.6.0.2624) in 0.26s


**Add Proteox to QCoDeS station and print readable snapshot of cryostat.**

In [3]:
station.add_component(Proteox)
Proteox.print_readable_snapshot()

Proteox:
	parameter                  value
--------------------------------------------------------------------------------
Cold_Plate_Temperature      :	0 (K)
IDN                         :	{'vendor': 'Oxford Instruments', 'model': 'oi:D...
Magnet_Current_Vector       :	(0.0, 0.0, -0.0116) (('A', 'A', 'A'))
Magnet_State                :	Holding Persistent 
Magnet_Temperature          :	813.1 (K)
Magnetic_Field_Vector       :	(0.0, 0.0, 0.0) (('T', 'T', 'T'))
Mixing_Chamber_Heater_Power :	0 (W)
Mixing_Chamber_Temperature  :	0 (K)
OVC_Pressure                :	2440 (Pa)
P1_Pressure                 :	87013 (Pa)
P2_Pressure                 :	171 (Pa)
P3_Pressure                 :	430 (Pa)
P4_Pressure                 :	33656 (Pa)
P5_Pressure                 :	34647 (Pa)
P6_Pressure                 :	451.9 (Pa)
PT1_Head_Temperature        :	294.3 (K)
PT1_Plate_Temperature       :	294.24 (K)
PT2_Head_Temperature        :	294.01 (K)
PT2_Plate_Temperature       :	294.03 (K)
Sorb_Temperature    

**Read Parameters**

In [4]:
Proteox.PT2_Plate_Temperature()

293.9055

In [5]:
Proteox.Magnetic_Field_Vector()

(0.0, 0.0, 0.0)

In [6]:
Proteox.Magnetic_Field_Vector()[2]

0.0

# Magnet Control

In [7]:
Proteox.Magnet_State()

'Holding Persistent'

**Set target field value and ramp rate.** 

````python
set_magnet_target(
  Coordinate_system : int, 
  X_feild_value : float, 
  Y_field_value : float, 
  Z_field_value : float, 
  Sweep_mode : str, 
  Sweep_rate : float 
)
````

Options for coordinate system are:  

  0 - Cartesian (X,Y,Z)  
  10 - Cylinderical (Rho, Phi, Z)  
  20 - Sphereical (R,Theta,Phi)  

For a solenoid magnet X_field_value and Y_field_value should be set to zero.

Options for Sweep_mode (and Sweep_rate) are: 
 
  'ASAP' (Sweep_rate is ignored. Can be any float)  
  'TIME' (Sweep_rate = time in minutes)   
  'RATE' (Sweep_rate = rate in T/min)   

In [3]:
#for z-solenoid magnet set target field to 0.1 T and sweep rate to 0.25 T/min.
Proteox.set_magnet_target(0,0,0,0.1,'RATE',0.25)

#If running with oi.DECS firmware =< 0.5.1, ingore the error "Error parsing response: Length of data record inconsistent with record type". The magnet field target should have still been set. Check the GUI.
#You will recieve this error because the data sent back from oi.DECS won't be handled correctly for firmware versions =< 0.5.1.

2024-01-18 13:23:01,578 - INFO - Error parsing response: Length of data record inconsistent with record type


Sweep the field. Then wait until Magnet status has gone back to 'Holding Not Persistent'.

In [ ]:
Proteox.sweep_field()
Proteox.wait_until_field_stable()

For high resolution field sweeps, set field as normal, then use the `sweep_small_field_step` function.

In [ ]:
Proteox.set_magnet_target(0,0,0,0.005,'ASAP',0.25)
Proteox.sweep_small_field_step('Z')

# You can only ramp one sub-group (X,Y or Z) at a time.
# When ramping the VRM subgroup the GUI will continue to show the VRM status as ‘Holding Not Persistent’ but the iPS will ramp the current to your field set point.
# The field value displayed on the GUI always rounds to 2 decimal places, but if you use the API to ask for the field value, it gives you the correct more precise value.

In [3]:
Proteox.close()

![image_test](./OI_images/nanoscience.png)